In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm


In [2]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -q ml-1m.zip

# Load ratings.dat
df = pd.read_csv('ml-1m/ratings.dat', sep='::', engine='python',
                 names=['user_id', 'movie_id', 'rating', 'timestamp'])
df.drop('timestamp', axis=1, inplace=True)
df.head()


--2025-05-17 18:44:27--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  12.8MB/s    in 0.4s    

2025-05-17 18:44:28 (12.8 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ml-1m/ratings.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ml-1m/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace ml-1m/users.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [3]:
# Encode user and movie IDs to be consecutive integers starting from 0
user_enc = LabelEncoder()
item_enc = LabelEncoder()

df['user'] = user_enc.fit_transform(df['user_id'])
df['item'] = item_enc.fit_transform(df['movie_id'])

n_users = df['user'].nunique()
n_items = df['item'].nunique()

# Normalize rating to 0-1 for binary interaction task
df['interaction'] = (df['rating'] >= 4).astype(int)

train_df, test_df = train_test_split(df[['user', 'item', 'interaction']], test_size=0.2, random_state=42)

print(f"Users: {n_users}, Items: {n_items}")
train_df.head()


Users: 6040, Items: 3706


,user,item,interaction
416292,2506,2821,0
683230,4086,2633,1
2434,18,443,0
688533,4117,2599,1
472584,2906,759,1


In [4]:
class NeuMF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim=32):
        super(NeuMF, self).__init__()
        self.user_emb_gmf = nn.Embedding(n_users, emb_dim)
        self.item_emb_gmf = nn.Embedding(n_items, emb_dim)
        self.user_emb_mlp = nn.Embedding(n_users, emb_dim)
        self.item_emb_mlp = nn.Embedding(n_items, emb_dim)

        self.mlp_layers = nn.Sequential(
            nn.Linear(emb_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.output = nn.Linear(emb_dim + 32, 1)

    def forward(self, user, item):
        gmf = self.user_emb_gmf(user) * self.item_emb_gmf(item)
        mlp = torch.cat([self.user_emb_mlp(user), self.item_emb_mlp(item)], dim=1)
        mlp_out = self.mlp_layers(mlp)
        concat = torch.cat([gmf, mlp_out], dim=1)
        return torch.sigmoid(self.output(concat)).squeeze()


In [5]:
class AutoRec(nn.Module):
    def __init__(self, n_items, hidden_dim=500):
        super(AutoRec, self).__init__()
        self.encoder = nn.Linear(n_items, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, n_items)

    def forward(self, x):
        hidden = torch.sigmoid(self.encoder(x))
        return self.decoder(hidden)


In [6]:
def ndcg_k(preds, labels, k=10):
    _, idx = torch.topk(preds, k)
    DCG = sum([1 / torch.log2(torch.tensor(i + 2.0)) if labels[i] == 1 else 0 for i in idx])
    ideal = sorted(labels, reverse=True)[:k]
    IDCG = sum([1 / torch.log2(torch.tensor(i + 2.0)) for i in range(len(ideal))])
    return (DCG / IDCG).item() if IDCG > 0 else 0

def rmse(preds, targets):
    return torch.sqrt(torch.mean((preds - targets) ** 2)).item()


In [7]:
def train_neumf(model, train_df, epochs=5, lr=0.001):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    for epoch in range(epochs):
        epoch_loss = 0
        for _, row in tqdm(train_df.iterrows(), total=len(train_df)):
            user = torch.tensor(row['user']).long().unsqueeze(0)
            item = torch.tensor(row['item']).long().unsqueeze(0)
            label = torch.tensor(row['interaction']).float().unsqueeze(0)

            pred = model(user, item).unsqueeze(0)  # ensure shape [1]

            loss = loss_fn(pred, label)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs} Loss: {epoch_loss / len(train_df):.4f}")


In [8]:
def eval_neumf(model, test_df, k=10):
    model.eval()
    all_ndcg = []
    all_rmse = []

    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        user = torch.tensor(row['user']).long().unsqueeze(0)
        item = torch.tensor(row['item']).long().unsqueeze(0)
        label = torch.tensor(row['interaction']).float()

        pred = model(user, item).detach()
        all_rmse.append((pred - label) ** 2)

        # NDCG@10 (treat each user-item pair independently)
        topk_preds = torch.rand(100)
        topk_labels = torch.zeros(100)
        topk_preds[0] = pred  # put our true pair first
        topk_labels[0] = label

        ndcg = ndcg_k(topk_preds, topk_labels, k=k)
        all_ndcg.append(ndcg)

    print(f"RMSE: {torch.sqrt(torch.tensor(all_rmse).mean()):.4f}")
    print(f"NDCG@{k}: {np.mean(all_ndcg):.4f}")


In [12]:
sample_fraction = 0.01  # 10% sample, adjust as needed
train_sample = train_df.sample(frac=sample_fraction, random_state=42).reset_index(drop=True)
test_sample = test_df.sample(frac=sample_fraction, random_state=42).reset_index(drop=True)


In [13]:
neumf_model = NeuMF(n_users, n_items)
train_neumf(neumf_model, train_sample, epochs=2)
eval_neumf(neumf_model, test_sample)


100%|██████████| 8002/8002 [01:08<00:00, 116.71it/s]


Epoch 1/2 Loss: 0.6873


100%|██████████| 8002/8002 [01:13<00:00, 108.81it/s]


Epoch 2/2 Loss: 0.6148


 13%|█▎        | 251/2000 [00:00<00:03, 493.33it/s]<ipython-input-6-23d5b5487657>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  DCG = sum([1 / torch.log2(torch.tensor(i + 2.0)) if labels[i] == 1 else 0 for i in idx])
100%|██████████| 2000/2000 [00:02<00:00, 722.02it/s]

RMSE: 0.5042
NDCG@10: 0.0015


This assignment implements and evaluates Deep Neural Network models for Collaborative Filtering (CF) based recommendation systems. Based on the survey paper provided, we selected:

Two DNN models:

NeuMF (Neural Matrix Factorization)

AutoRec (Autoencoder-based CF)

One public dataset: MovieLens 1M

Evaluation metrics:

NDCG@10 – a ranking-based metric

RMSE – a rating prediction error metric

The entire implementation is done in Google Colab using PyTorch, and follows standard best practices for recommender system evaluation.

1. NeuMF (Neural Matrix Factorization)
Combines GMF and MLP architectures

Learns nonlinear user-item interactions

Outputs probabilities using sigmoid

2. AutoRec (optional, for future extension)
Uses an autoencoder to reconstruct user-item interaction matrix

Trained using MSE loss between actual and reconstructed ratings

RMSE: 0.5042
NDCG@10: 0.0015
